In [1]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Nombres de los archivos parquet
archivos_parquet = ['clean_df_ur.parquet', 'definitivo_df_ui.parquet', 'definitivo_df_sg.parquet']

# Cargar el primer archivo parquet
df_ur = pd.read_parquet('DataSet/' + archivos_parquet[0])

# Cargar el segundo archivo parquet
df_ui = pd.read_parquet('DataSet/' + archivos_parquet[1])

# Cargar el tercer archivo parquet
df_sg = pd.read_parquet('DataSet/' + archivos_parquet[2])

A continuación, se crea el modelo de recomendación item-item. Para ello se unen los dataframes df_ui y df_sg para crear un conjunto de datos que incluye item_id, el logaritmo del tiempo de juego (playtime_forever_log), y el precio (price) de cada juego.

Se eligen playtime_forever_log y price como las características principales para el modelo. Estas características son fundamentales para entender las preferencias de los usuarios y la popularidad de los juegos como se justificó en el EDA.

Se aplica PCA para reducir la dimensionalidad de los datos a dos componentes principales. Esto ayuda a simplificar el modelo y a centrarse en las características más significativas.

Finalmente, se define una función "recomendacion_juego" que toma un ID de juego y devuelve una lista de recomendaciones. La función utiliza la similitud del coseno para encontrar juegos con características similares en el espacio reducido.

Este modelo ofrece una forma efectiva de recomendar juegos basándose en patrones de juego y precios, lo que puede ser especialmente útil para plataformas de distribución de juegos o servicios de recomendación personalizada.

In [3]:
# Unir df_ui y df_sg para obtener características relevantes de juegos
combined_df = pd.merge(df_ui[['item_id', 'playtime_forever_log']], 
                       df_sg[['id', 'price']], 
                       left_on='item_id', 
                       right_on='id')

# Seleccionar playtime_forever_log y price como características para el modelo
game_features = combined_df[['playtime_forever_log', 'price']]

# Inicializar PCA para reducir la dimensionalidad a dos componentes
pca = PCA(n_components=2)  

# Aplicar PCA a las características seleccionadas
game_features_reduced = pca.fit_transform(game_features)

# Definir la función de recomendación de juegos
def recomendacion_juego(game_id, game_features_reduced, df_ui, top_n=5):
    # Verificar si el ID del juego está en el rango válido
    if game_id not in range(len(game_features_reduced)):
        return "El ID ingresado no arroja resultados. Inténtelo de nuevo."

    # Extraer el vector de características para el juego especificado
    game_vector = game_features_reduced[game_id:game_id+1]

    # Calcular la similitud del coseno entre el juego y todos los demás
    cosine_similarities = cosine_similarity(game_vector, game_features_reduced)

    # Ordenar y seleccionar los índices de los juegos más similares
    similar_games_indices = cosine_similarities.argsort()[0][-top_n-1:-1][::-1]

    # Excluir el juego actual de las recomendaciones
    similar_games_indices = [i for i in similar_games_indices if i != game_id]

    # Verificar si se encontraron juegos similares
    if len(similar_games_indices) == 0:
        return "No se encontraron juegos similares para recomendar."

    # Obtener los nombres de los juegos recomendados
    recommended_games = df_ui.loc[similar_games_indices, 'item_name']

    # Devolver un mensaje con los nombres de los juegos recomendados por el modelo
    return f"Según el ID ingresado, se recomiendan los siguientes juegos: {recommended_games[:top_n].tolist()}"

# Ejemplo de uso del modelo para obtener recomendaciones
id_del_juego = 227300  
print(recomendacion_juego(id_del_juego, game_features_reduced, df_ui))



Según el ID ingresado, se recomiendan los siguientes juegos: ['Kerbal Space Program', "Tom Clancy's Ghost Recon Phantoms - NA", 'Guncraft', 'Tropico 5', 'VVVVVV']
